In [1]:
import pandas as pd
import os
import math
import numpy as np
import cx_Oracle
from config import oracle_wfm_key
import time
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.types import String
from tqdm import tqdm
import itertools
import databases as db
from decimal import Decimal, getcontext
from math import ceil, exp, floor

In [2]:
ods = db.oracle('cco_wfm',oracle_wfm_key)
verint  = db.ssms()
service = ['BSSC Billing ALL (new)','BSSC Senior','Commercial COAST','Mooresville_BSSC_Billing','BSSC Sales Cable','BSSC Sales Wireline','Mooresville_BSSC_Sales',
            'Bend/Cable Consumer Sales','OB_QCB Bend/Cable Sales','Cable Specialist','OB_OCB_Cable_Con_Spec','Cable Sales Seniors',
            'Financial Services Res','Financial Services Bus',
            'Wireline Sales and Service','Wireline Service QCB','Wireline Consumer Specialist','Wireline Consumer Triage','Wireline Retention QCB','Wireline Senior Specialist 1']
start = '2023-01-01 00:00:00.000'
end = '2023-03-31 23:45:00.000'
appended_data = []
for name in service:
    new_query = f'''SELECT 
                        [Queue],
                        [DateTime],
                        [Actual_CV],
                        [Forecasted_CV],
                        [Actual_AHT],
                        [Forecasted_AHT]                                             
                    FROM [BPMAINDB].[dbo].[V_AdHoc_PerformanceStatistics]
                    WHERE ([Queue] = '{name}') AND ([DateTime] BETWEEN '{start}' AND '{end}')
                    AND ([UserName] = 'satverintwrkoptmgmt')'''
    data = pd.read_sql(new_query, verint)
    appended_data.append(data)
appended_data = pd.concat(appended_data)
appended_data.tail(3)

,Queue,DateTime,Actual_CV,Forecasted_CV,Actual_AHT,Forecasted_AHT
8633,Wireline Senior Specialist 1,2023-03-31 23:15:00,0.0,NaN,0.0,NaN
8634,Wireline Senior Specialist 1,2023-03-31 23:30:00,0.0,NaN,0.0,NaN
8635,Wireline Senior Specialist 1,2023-03-31 23:45:00,0.0,NaN,0.0,NaN


In [3]:
team_dict = {'BSSC Billing ALL (new)' : 'Commercial_Billing','BSSC Senior' : 'Commercial_Billing','Commercial COAST' : 'Commercial_Billing','Mooresville_BSSC_Billing' : 'Commercial_Billing',
        'BSSC Sales Cable' : 'Commercial_Sales','BSSC Sales Wireline' : 'Commercial_Sales','Mooresville_BSSC_Sales' : 'Commercial_Sales',
        'Bend/Cable Consumer Sales' : 'Cable_Sales','OB_QCB Bend/Cable Sales' : 'Cable_Sales','Cable Specialist' : 'Cable_Retention','OB_OCB_Cable_Con_Spec' : 'Cable_Retention',
        'Cable Sales Seniors' : 'Cable_Seniors',
        'Financial Services Res' : 'Residential_FS','Financial Services Bus' : 'Commercial_FS',
        'Wireline Sales and Service' : 'Wireline_Sales','Wireline Service QCB' : 'Wireline_Sales',
        'Wireline Consumer Specialist' : 'Wireline_Retention','Wireline Consumer Triage' : 'Wireline_Retention','Wireline Retention QCB' : 'Wireline_Retention',
        'Wireline Senior Specialist 1': 'Wireline_Seniors'}
appended_data['Team'] = appended_data['Queue'].map(team_dict)
appended_data = appended_data.reindex(columns=['Team'] + list(appended_data.columns[:-1]))
appended_data.tail(3)

,Team,Queue,DateTime,Actual_CV,Forecasted_CV,Actual_AHT,Forecasted_AHT
8633,Wireline_Seniors,Wireline Senior Specialist 1,2023-03-31 23:15:00,0.0,NaN,0.0,NaN
8634,Wireline_Seniors,Wireline Senior Specialist 1,2023-03-31 23:30:00,0.0,NaN,0.0,NaN
8635,Wireline_Seniors,Wireline Senior Specialist 1,2023-03-31 23:45:00,0.0,NaN,0.0,NaN


In [4]:
appended_data['Actual_Workload'] = appended_data['Actual_AHT'] * appended_data['Actual_CV']
appended_data['Actual_Workload'] = appended_data['Actual_Workload'].replace(0, np.nan)
appended_data['Forecasted_Workload'] = appended_data['Forecasted_AHT'] * appended_data['Forecasted_CV']
appended_data['Forecasted_Workload'] = appended_data['Forecasted_Workload'].replace(0, np.nan)

In [5]:
team = ['Commercial_Billing','Commercial_Sales','Cable_Sales','Cable_Retention','Cable_Seniors','Residential_FS','Commercial_FS',
        'Wireline_Sales','Wireline_Retention','Wireline_Seniors']
dfs = []
for t in team:
    team_group = appended_data.loc[(appended_data['Team']  == t)]
    actual_cv = team_group.groupby(['Team','DateTime'])['Actual_CV'].sum()
    actual_cv = actual_cv.reset_index()
    actual_cv.set_index('DateTime', inplace=True) # set the index to the 'DateTime' column
    actual_cv = actual_cv.resample('30 min').sum() # remove the 'on' parameter
    actual_cv.reset_index(inplace=True)
    actual_cv.set_index('DateTime', inplace=True)
    
    forecasted_cv = team_group.groupby(['Team','DateTime'])['Forecasted_CV'].sum()
    forecasted_cv = forecasted_cv.reset_index()
    forecasted_cv.set_index('DateTime', inplace=True) # set the index to the 'DateTime' column
    forecasted_cv = forecasted_cv.resample('30 min').sum() # remove the 'on' parameter
    forecasted_cv.reset_index(inplace=True)
    forecasted_cv.set_index('DateTime', inplace=True)
    
    grouped_aht = team_group.groupby(['Team','DateTime'])['Actual_Workload'].sum()
    grouped_aht = grouped_aht.reset_index()
    grouped_aht['Actual_Workload'] = grouped_aht['Actual_Workload'].replace(0, np.nan)
    grouped_aht.set_index('DateTime', inplace=True) # set the index to the 'DateTime' column
    grouped_aht = grouped_aht.resample('30 min').sum() # remove the 'on' parameter
    grouped_cv = team_group.groupby(['Team','DateTime'])['Actual_CV'].sum()
    grouped_cv = grouped_cv.reset_index()
    grouped_cv.set_index('DateTime', inplace=True) # set the index to the 'DateTime' column
    grouped_cv = grouped_cv.resample('30 min').sum() # remove the 'on' parameter
    actual_aht = grouped_aht.merge(grouped_cv, on='DateTime', how='left')
    actual_aht['Actual_AHT'] = actual_aht['Actual_Workload'] / actual_aht['Actual_CV']
    actual_aht = actual_aht.drop(columns=['Actual_Workload','Actual_CV'])
    actual_aht.reset_index(inplace=True)
    actual_aht.set_index('DateTime', inplace=True)
    actual_aht['Actual_AHT'] = actual_aht['Actual_AHT'].fillna(0)
    actual_aht['Actual_AHT'] = round(actual_aht['Actual_AHT'])
    
    grouped_faht = team_group.groupby(['Team','DateTime'])['Forecasted_Workload'].sum()
    grouped_faht = grouped_faht.reset_index()
    grouped_faht['Forecasted_Workload'] = grouped_faht['Forecasted_Workload'].replace(0, np.nan)
    grouped_faht.set_index('DateTime', inplace=True) # set the index to the 'DateTime' column
    grouped_faht = grouped_faht.resample('30 min').sum() # remove the 'on' parameter
    grouped_fcv = team_group.groupby(['Team','DateTime'])['Forecasted_CV'].sum()
    grouped_fcv = grouped_fcv.reset_index()
    grouped_fcv.set_index('DateTime', inplace=True) # set the index to the 'DateTime' column
    grouped_fcv = grouped_fcv.resample('30 min').sum() # remove the 'on' parameter
    forecasted_aht = grouped_faht.merge(grouped_fcv, on='DateTime', how='left')
    forecasted_aht['Forecasted_AHT'] = forecasted_aht['Forecasted_Workload'] / forecasted_aht['Forecasted_CV']
    forecasted_aht = forecasted_aht.drop(columns=['Forecasted_Workload','Forecasted_CV'])
    forecasted_aht.reset_index(inplace=True)
    forecasted_aht.set_index('DateTime', inplace=True)
    forecasted_aht['Forecasted_AHT'] = forecasted_aht['Forecasted_AHT'].fillna(0)
    forecasted_aht['Forecasted_AHT'] = round(forecasted_aht['Forecasted_AHT'])
    
    cv = actual_cv.merge(forecasted_cv, on=['DateTime'], how='left')
    aht = actual_aht.merge(forecasted_aht, on=['DateTime'], how='left')
    table = cv.merge(aht, on=['DateTime'], how='left')
    table['Team'] = t
    table = table.reindex(columns=['Team'] + list(table.columns[:-1]))
    dfs.append(table)

result = pd.concat(dfs)
result.head()


,Team,Actual_CV,Forecasted_CV,Actual_AHT,Forecasted_AHT
DateTime,,,,,
2023-01-01 00:00:00,Commercial_Billing,0.0,0.0,0.0,0.0
2023-01-01 00:30:00,Commercial_Billing,0.0,0.0,0.0,0.0
2023-01-01 01:00:00,Commercial_Billing,0.0,0.0,0.0,0.0
2023-01-01 01:30:00,Commercial_Billing,0.0,0.0,0.0,0.0
2023-01-01 02:00:00,Commercial_Billing,0.0,0.0,0.0,0.0


In [6]:
result.reset_index(inplace=True)
result['Date'] = result['DateTime'].dt.date
result['Time'] = result['DateTime'].dt.time
result = result.drop('DateTime', axis=1)
result['Time_Interval'] = '00:30'
result = result[['Team', 'Date', 'Time', 'Time_Interval', 'Actual_CV', 'Forecasted_CV', 'Actual_AHT', 'Forecasted_AHT']]
result.head()

,Team,Date,Time,Time_Interval,Actual_CV,Forecasted_CV,Actual_AHT,Forecasted_AHT
0,Commercial_Billing,2023-01-01,00:00:00,00:30,0.0,0.0,0.0,0.0
1,Commercial_Billing,2023-01-01,00:30:00,00:30,0.0,0.0,0.0,0.0
2,Commercial_Billing,2023-01-01,01:00:00,00:30,0.0,0.0,0.0,0.0
3,Commercial_Billing,2023-01-01,01:30:00,00:30,0.0,0.0,0.0,0.0
4,Commercial_Billing,2023-01-01,02:00:00,00:30,0.0,0.0,0.0,0.0


In [7]:
# pass all variables from the tuple
team_settings = {
    'Commercial_Billing': (0.60, 0.05, 30, 30),
    'Commercial_Sales': (0.60, 0.05, 30, 30),
    'Cable_Sales': (0.65, 0.05, 30, 30),
    'Cable_Retention': (0.65, 0.05, 30, 30),
    'Cable_Seniors': (0.70, 0.05, 30, 30),
    'Residential_FS': (0.65, 0.05, 30, 30),
    'Commercial_FS': (0.65, 0.05, 30, 30),
    'Wireline_Sales': (0.65, 0.10, 30, 30),
    'Wireline_Retention': (0.65, 0.10, 30, 30),
    'Wireline_Seniors': (0.70, 0.05, 30, 30)
}


# # set the constant variables
# Reporting_Period = 30
# service_level_time = 30

## Erlang Testing

In [8]:
class ErlangC:
    """
    Computes the number of positions required to attend a number of transactions in a
    queue system based on erlangc.rst. Implementation inspired on:
    https://lucidmanager.org/data-science/call-centre-workforce-planning-erlang-c-in-r/
    Parameters
    ----------
    transactions: float,
        The number of total transactions that comes in an interval.
    aht: float,
        Average handling time of a transaction (minutes).
    asa: float,
        The required average speed of answer (minutes).
    interval: int,
        Interval length (minutes) where the transactions come in
    shrinkage: float,
        Percentage of time that an operator unit is not available.
    """

    def __init__(self, transactions: float, aht: float, asa: float,
                 interval: int, shrinkage=0.0,
                 **kwargs):

        if transactions <= 0:
            raise ValueError("transactions can't be smaller or equals than 0")

        if aht <= 0:
            raise ValueError("aht can't be smaller or equals than 0")

        if asa <= 0:
            raise ValueError("asa can't be smaller or equals than 0")

        if interval <= 0:
            raise ValueError("interval can't be smaller or equals than 0")

        if shrinkage < 0 or shrinkage >= 1:
            raise ValueError("shrinkage must be between in the interval [0,1)")

        self.n_transactions = transactions
        self.aht = aht / 60  # Convert aht from seconds to minutes
        self.interval = interval
        self.asa = asa
        self.intensity = (self.n_transactions / self.interval) * self.aht
        self.shrinkage = shrinkage


    def waiting_probability(self, positions: int, scale_positions: bool = False):
        """
        Returns the probability of waiting in the queue
        Parameters
        ----------
        positions: int,
            The number of positions to attend the transactions.
        scale_positions: bool, default=False
            Set it to True if the positions were calculated using shrinkage.
        """

        if scale_positions:
            productive_positions = floor((1 - self.shrinkage) * positions)
        else:
            productive_positions = positions

        erlang_b_inverse = 1
        for position in range(1, productive_positions + 1):
            erlang_b_inverse = 1 + (erlang_b_inverse * position / self.intensity)

        erlang_b = 1 / erlang_b_inverse
        return productive_positions * erlang_b / (productive_positions - self.intensity * (1 - erlang_b))

    def service_level(self, positions: int, scale_positions: bool = False):
        """
        Returns the expected service level given a number of positions
        Parameters
        ----------
        positions: int,
            The number of positions attending.
        scale_positions: bool, default = False
            Set it to True if the positions were calculated using shrinkage.
        """
        if scale_positions:
            productive_positions = floor((1 - self.shrinkage) * positions)
        else:
            productive_positions = positions

        probability_wait = self.waiting_probability(productive_positions, scale_positions=False)
        exponential = exp(-(productive_positions - self.intensity) * (self.asa / self.aht))
        return max(0, 1 - (probability_wait * exponential))

    def achieved_occupancy(self, positions: int, scale_positions: bool = False):
        """
        Returns the expected occupancy of positions
        Parameters
        ----------
        positions: int,
            The number of raw positions
        scale_positions: bool, default=False
            Set it to True if the positions were calculated using shrinkage.
        """
        if scale_positions:
            productive_positions = floor((1 - self.shrinkage) * positions)
        else:
            productive_positions = positions

        return self.intensity / productive_positions

    def required_positions(self, service_level: float, max_occupancy: float = 1.0):
        """
        Computes the requirements using erlangc.rst
        Parameters
        ----------
        service_level: float,
            Target service level
        max_occupancy: float,
            The maximum fraction of time that a transaction can occupy a position
        Returns
        -------
        raw_positions: int,
            The required positions assuming shrinkage = 0
        positions: int,
            The number of positions needed to ensure the required service level
        service_level: float,
            The fraction of transactions that are expected to be assigned to a position,
            before the asa time
        occupancy: float,
            The expected occupancy of positions
        waiting_probability: float,
            The probability of a transaction waiting in the queue
        """

        if service_level < 0 or service_level > 1:
            raise ValueError("service_level must be between 0 and 1")

        if max_occupancy < 0 or max_occupancy > 1:
            raise ValueError("max_occupancy must be between 0 and 1")

        positions = round(self.intensity + 1)
        achieved_service_level = self.service_level(positions, scale_positions=False)
        while achieved_service_level < service_level:
            positions += 1
            achieved_service_level = self.service_level(positions, scale_positions=False)

        achieved_occupancy = self.achieved_occupancy(positions, scale_positions=False)

        raw_positions = ceil(positions)

        if achieved_occupancy > max_occupancy:
            raw_positions = ceil(self.intensity / max_occupancy)
            achieved_occupancy = self.achieved_occupancy(raw_positions)
            achieved_service_level = self.service_level(raw_positions)

        waiting_probability = self.waiting_probability(positions=raw_positions)
        positions = ceil(raw_positions / (1 - self.shrinkage))

        return {"raw_positions": raw_positions,
                "positions": positions,
                "service_level": achieved_service_level,
                "occupancy": achieved_occupancy,
                "waiting_probability": waiting_probability}

In [9]:
# Loop through the DataFrame
for index, row in result.iterrows():
    team = row['Team']
    shrinkage, service_level_percent = team_settings[team]

    forecasted_cv = row['Forecasted_CV']
    forecasted_aht = row['Forecasted_AHT']
    actual_cv = row['Actual_CV']
    actual_aht = row['Actual_AHT']

    required_fte_forecast = calculate_agents_v2(forecasted_cv, Reporting_Period, forecasted_aht, service_level_percent, service_level_time, shrinkage)
    required_fte_actual = calculate_agents_v2(actual_cv, Reporting_Period, actual_aht, service_level_percent, service_level_time, shrinkage)

    result.at[index, 'Required_FTE_Forecast'] = required_fte_forecast
    result.at[index, 'Required_FTE_Actual'] = required_fte_actual

ValueError: too many values to unpack (expected 2)

In [ ]:
directory = "I:\\GitHub\\forecasting\\accuracy\\accuracy_exports\\"
result['Date'] = pd.to_datetime(result['Date'])
result['Month'] = result['Date'].dt.strftime('%Y-%m')
teams = result['Team'].unique()
for team in teams:
    team_result = result[result['Team'] == team]
    for month in team_result['Month'].unique():
        month_result = team_result[team_result['Month'] == month]
        file_name = f"{team}_{month}.txt"
        file_path = os.path.join(directory, file_name)
        month_result.to_csv(file_path, sep='\t', index=False)

In [ ]:
# Sample data
sample_data = [
    (4, 343),
    (5, 725),
    (10, 469),
    (14, 394),
    (9, 498),
    (16, 638),
    (16, 659),
    (14, 748),
    (6, 713),
    (17, 451),
    (15, 666),
    (14, 872),
    (15, 539),
    (11, 527),
    (15, 602),
    (12, 658),
    (22, 829),
    (18, 533),
    (10, 442),
    (2, 570),
    (7, 243),
    (2, 1089),
    (1, 711),
]

# Constants
service_level_percent = 0.60
shrinkage = 0.05
reporting_period = 30
service_level_time = 30

# Collect the results in a list
results = []
for calls, handle_time in sample_data:
    erlang_c = ErlangC(calls, reporting_period, handle_time, service_level_time, shrinkage)
    required_positions_data = erlang_c.required_positions(service_level_percent)
    results.append((calls, handle_time, *required_positions_data.values()))

# Create a DataFrame with the results
column_names = ['calls', 'handle_time', 'raw_positions', 'positions', 'service_level', 'occupancy', 'waiting_probability']
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
df


In [ ]:
class ErlangC:
    def __init__(self, calls, reporting_period, aht, service_level_time, shrinkage):
        self.calls = calls
        self.reporting_period = reporting_period
        self.aht = aht
        self.service_level_time = service_level_time
        self.shrinkage = shrinkage
        self.intensity = self.traffic_intensity()

    def traffic_intensity(self):
        return (self.calls * self.aht) / self.reporting_period

    @staticmethod
    def erlang_c_formula(intensity, positions):
        getcontext().prec = 50
        decimal_intensity = Decimal(intensity)
        decimal_positions = Decimal(positions)
        numerator = (decimal_intensity ** decimal_positions) / Decimal(math.factorial(positions))
        denominator = Decimal(0)
        for k in range(0, positions):
            denominator += (decimal_intensity ** Decimal(k)) / Decimal(math.factorial(k))

        if decimal_positions == decimal_intensity:
            return 1.0
        
        denominator += (numerator * (decimal_positions / (decimal_positions - decimal_intensity)))
        return float(numerator / denominator)

    def waiting_probability(self, positions, scale_positions=True):
        if scale_positions:
            productive_positions = ceil(positions * (1 - self.shrinkage))
        else:
            productive_positions = positions

        return ErlangC.erlang_c_formula(self.intensity, productive_positions)

    def service_level(self, positions, scale_positions=True):
        if scale_positions:
            positions = ceil(positions / (1 - self.shrinkage))
        probability_wait = self.waiting_probability(positions, scale_positions=False)
        exponential = exp(-(positions - self.intensity) * (self.service_level_time / self.aht))
        return max(0, 1 - probability_wait * exponential)


    def occupancy(self, positions, scale_positions=True):
        if scale_positions:
            productive_positions = ceil(positions * (1 - self.shrinkage))
        else:
            productive_positions = positions

        return self.intensity / productive_positions

    def required_positions(self, service_level, max_occupancy=1.0):
        positions = ceil(self.intensity)
        achieved_service_level = self.service_level(positions, scale_positions=False)

        while achieved_service_level < service_level:
            positions += 1
            achieved_service_level = self.service_level(positions, scale_positions=False)

        self.positions = positions
        achieved_service_level = self.service_level(positions, scale_positions=True)  # Adjusted
        achieved_occupancy = self.occupancy(positions)

        return {
            "raw_positions": positions,
            "positions": ceil(positions / (1 - self.shrinkage)),
            "service_level": achieved_service_level,
            "occupancy": achieved_occupancy,
            "waiting_probability": self.waiting_probability(positions, scale_positions=False),
        }


def calculate_agents_v2(calls, reporting_period, average_handling_time, service_level_percent, service_level_time, shrinkage):
    if math.isnan(calls) or math.isnan(reporting_period) or math.isnan(average_handling_time) or math.isnan(service_level_percent) or math.isnan(service_level_time) or math.isnan(shrinkage) or calls == 0 or average_handling_time == 0:
        return float('nan')

    if not (0 < calls < float('inf')) or not (0 < reporting_period < float('inf')) or not (0 < average_handling_time < float('inf')) or not (0 <= service_level_percent <= 1) or not (0 < service_level_time < float('inf')) or not (0 <= shrinkage < 1):
        raise ValueError(f"Invalid input values: calls={calls}, reporting_period={reporting_period}, average_handling_time={average_handling_time}, service_level_percent={service_level_percent}, service_level_time={service_level_time}, shrinkage={shrinkage}")

    asa = service_level_time / 60  # Convert seconds to minutes
    interval = reporting_period * 60  # Convert hours to minutes

    erlang_c_instance = ErlangC(transactions=calls, aht=average_handling_time, asa=asa, interval=interval, shrinkage=shrinkage)
    required_positions_data = erlang_c_instance.required_positions(service_level=service_level_percent)

    # print(f"calls: {calls}, reporting_period: {reporting_period}, average_handling_time: {average_handling_time}, service_level_percent: {service_level_percent}, service_level_time: {service_level_time}, shrinkage: {shrinkage}")
    # print(f"asa: {asa}, interval: {interval}")
    # print(f"required_positions_data: {required_positions_data}")

    return required_positions_data['positions']

In [ ]:
# Sample data
sample_data = [
    (4, 343),
    (5, 725),
    (10, 469),
    (14, 394),
    (9, 498),
    (16, 638),
    (16, 659),
    (14, 748),
    (6, 713),
    (17, 451),
    (15, 666),
    (14, 872),
    (15, 539),
    (11, 527),
    (15, 602),
    (12, 658),
    (22, 829),
    (18, 533),
    (10, 442),
    (2, 570),
    (7, 243),
    (2, 1089),
    (1, 711),
]

# Constants
service_level_percent = 0.60
shrinkage = 0.05
reporting_period = 30
service_level_time = 30

# Collect the results in a list
results = []
for calls, handle_time in sample_data:
    erlang_c = ErlangC(calls, reporting_period, handle_time, service_level_time, shrinkage)
    required_positions_data = erlang_c.required_positions(service_level_percent)
    results.append((calls, handle_time, *required_positions_data.values()))

# Create a DataFrame with the results
column_names = ['calls', 'handle_time', 'raw_positions', 'positions', 'service_level', 'occupancy', 'waiting_probability']
df = pd.DataFrame(results, columns=column_names)

# Display the DataFrame
df